In [1]:
import streamlit as st
from llama_index import VectorStoreIndex, ServiceContext
from langchain.llms import OpenAI
from llama_index import SimpleDirectoryReader
from llama_index.readers import SimpleWebPageReader
from langchain.embeddings import SentenceTransformerEmbeddings
from llama_index import ServiceContext
from langchain.llms import OpenAI
from langchain.embeddings import SentenceTransformerEmbeddings
from llama_index import SimpleDirectoryReader
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import re
import logging
import os
import requests
import nltk
from llama_index import Document
import pickle

In [34]:
import re
pattern = r'\d+\%'  # This pattern matches one or more digits, followed by % sign. Adjust the number of digits as needed (e.g., '\d{3}%' for exactly three digits).
string = "Your string here 40%"
match = re.search(pattern, string)
if match:  # If a pattern is found in the given string...
    print("Pattern '%s' was found at position %i." % (match.group(), match.start()))
else:   # Otherwise...
    print("No matching pattern was found.")


Pattern '40%' was found at position 17.


In [14]:
import random

def generate_random_list():
    list_size = random.randint(0, 4)
    random_list = [random.randint(1, 100) for _ in range(list_size)]
    return random_list


def calculate_average(numbers):
    sum_of_numbers = 0
    count = 0
    for number in numbers:
        sum_of_numbers += number
        count += 1
    # Intentional error: Forgot to handle the case when the input list is empty
    average = sum_of_numbers / count
    print(average)
    return average


for _ in range(20):  
    random_numbers = generate_random_list()
    calculate_average(random_numbers)

53.0
79.5


ZeroDivisionError: division by zero

In [28]:
def factorial(n):
    if n == 0: # Base Case, return 1 when n is zero since a factoiral of zero equals one (according to mathematical definition)
        return 1
    else:       # Recursive case, multiply the current number by the result from recursively calling this function with 'n-1' as argument.
        return n * factorial(n - 1)

factorial(3)

6

In [2]:
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_all_links_recursive(url, depth=3, token=None):
    headers = {'Authorization': f'access_token  {token}'}
    if depth == 0:
        return []
    try:
        response = requests.get(url, headers=headers, verify=True)  # Ignore SSL certificate verification
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        absolute_links = [urljoin(url, link['href']) for link in links]
        # Print links for the current page
        print(f"Links from {url}:")
        for link in absolute_links:
            print(link)
        # Recursively get links from child pages
        child_links = []
        for link in absolute_links:
            try:
                child_links.extend(get_all_links_recursive(link, depth=depth-1))
            except requests.exceptions.RequestException as e:
                print(f"Error retrieving content from {link}: {e}")

        return absolute_links + child_links
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving content from {url}: {e}")
    return []

In [7]:
response

<Response [200]>

In [15]:
import requests
token = 'NjAyNzg1NTg5MTkwOrJDgTp2htHj2J7ZNdMdTKSOWVuX'
headers={'Authorization': f'Bearer {token}' }
response = requests.get('https://wiki.devops.kratosdefense.com/display/EPOCH/EPOCH+IPS+Home', headers=headers)
response.content

b'    \n\n<!DOCTYPE html>\n<html lang="en-GB" >\n<head>\n                            <title>EPOCH IPS Home - EPOCH IPS - Confluence</title>\n    \n        \n\n                        \n    \n                        \n    \n\n                \n    \n    <meta http-equiv="X-UA-Compatible" content="IE=EDGE,chrome=IE7">\n<meta charset="UTF-8">\n<meta id="confluence-context-path" name="confluence-context-path" content="">\n<meta id="confluence-base-url" name="confluence-base-url" content="https://wiki.devops.kratosdefense.com">\n\n    <meta id="atlassian-token" name="atlassian-token" content="4aa009175828374fa82c90920f768b7ea182fa2d">\n\n\n<meta id="confluence-space-key" name="confluence-space-key" content="EPOCH">\n<script type="text/javascript">\n        var contextPath = \'\';\n</script>\n\n    \n\n    <meta name="confluence-request-time" content="1705678941152">\n        \n    \n        \n            <meta name="ajs-show-space-welcome-dialog" content="true">\n            <style>.ia-fixe

In [ ]:
def load_data():
    with st.spinner(text="Loading and indexing the kratos docs – hang tight! This should take 1-2 minutes."):
        if os.listdir("./data"):
            reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
            docs = reader.load_data()
        else:
            docs = list()
        with open('docs.pkl', 'rb') as f:
            new_docs = pickle.load(f)
        docs += new_docs
        service_context = ServiceContext.from_defaults(llm=llm,
                                                       embed_model=local_embeddings)
        index = VectorStoreIndex.from_documents(docs, service_context=service_context)
        return index

In [ ]:
local_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

llm = OpenAI(base_url="http://192.168.48.33:1234/v1",
             openai_api_key="not-needed",
             temperature=0.1,
             top_p=0.1,
             max_tokens=256,
             presence_penalty=1.2,
             frequency_penalty=1.2,
             streaming=False,
             # system_prompt="You are an expert on the Streamlit Python library and your job is to answer technical "
             #               "questions. Assume that all questions are related to the Streamlit Python library."
             #               " Keep your answers technical and based on facts – do not hallucinate features."
             )



reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
docs = reader.load_data()
for doc in docs:
    if doc.metadata['file_path'] == 'data\\skyminer-development.pdf':
        doc.metadata['url'] = "http://192.168.48.22:8082/repository/skyminer-dev/dev-env/skyminer-development.pdf#page=" + doc.metadata['page_label']
docs

In [ ]:
docs[0].metadata['file_path'] == 'data\\skyminer-development.pdf'

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
local_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

llm = OpenAI(base_url="http://192.168.48.33:1234/v1",
             openai_api_key="not-needed",
             temperature=0.1,
             top_p=0.1,
             max_tokens=256,
             presence_penalty=1.2,
             frequency_penalty=1.2,
             streaming=False,
             # system_prompt="You are an expert on the Streamlit Python library and your job is to answer technical "
             #               "questions. Assume that all questions are related to the Streamlit Python library."
             #               " Keep your answers technical and based on facts – do not hallucinate features."
             )

reader = SimpleWebPageReader(html_to_text=True,
                             metadata_fn=lambda url: {'url': url, 'file_name': url.split('/')[-1].split('%2F')[-1]})
docs = reader.load_data(['http://192.168.48.22:8082/repository/skyminer-dev/dev-env/skyminer-development.pdf'])
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=local_embeddings)
index = VectorStoreIndex.from_documents(docs, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is skyminer?")
response

In [ ]:
from llama_index import Document
from llama_index.schema import MetadataMode

new_doc = Document(
    text="""
        The event manager helps the user in this task by automatically duplicating specified metrics and adding the relevant time overrides, aliases and time aligns.
        """,
    metadata={
        "file_name": "event-manager.html",
        "url": "http://192.168.48.101/user-manual/query-ui/event-manager.html#overview",
        "title": "Event manager",
        "sub_title": "Overview"
    },
    excluded_llm_metadata_keys=["file_name", "url"],
    excluded_embed_metadata_keys = ["file_name", "url"],
    metadata_seperator="::",
    metadata_template="{key}=>{value}",
    text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
)

In [ ]:
reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
docs = reader.load_data()
docs = docs[-3:]
docs.append(new_doc)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=local_embeddings)
index = VectorStoreIndex.from_documents(docs, service_context=service_context)


In [ ]:
docs = reader.load_data()
docs

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("what is the event manager?")
print(response.response)
for node in response.source_nodes:
    if 'url' in node.metadata.keys():
        print(node.metadata['url'])
    else:
        print(node.metadata['file_path'])


In [ ]:
response

In [ ]:
docs_ = docs[-3:]
docs.append(new_doc)

In [ ]:
docs[10]

In [ ]:
docs[-1]

In [ ]:
index.ServiceContext

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is skyminer?")
response

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://192.168.48.29/query/api-documentation/features"
import requests
from bs4 import BeautifulSoup

def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text()
    return text, soup

# Example use
text, soup = extract_text(url)
print(soup)

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Replace 'your_url' with the URL of the web page you want to scrape
url = 'http://192.168.48.29/query/api-documentation/features'

# Create a Chrome webdriver with the help of ChromeDriverManager
driver = webdriver.Chrome()

# Navigate to the URL
driver.get(url)

# Wait for some time to let the page load (you may need to adjust this based on the page)
driver.implicitly_wait(30)

# Extract all text from the page
# Extract all visible text from the page
all_text_elements = driver.find_elements("//*[not(self::script) and not(self::style)]")
all_text = ' '.join([element.text for element in all_text_elements])

# Print or do whatever you want with the extracted text
print(all_text)

# Close the browser window
driver.quit()

In [ ]:
from llama_index import SummaryIndex
from llama_index.readers import SimpleWebPageReader
from IPython.display import Markdown, display
import os

llm = OpenAI(base_url="http://192.168.48.33:1234/v1",
             openai_api_key="not-needed",
             temperature=0.1,
             top_p=0.1,
             max_tokens=256,
             presence_penalty=1.2,
             frequency_penalty=1.2,
             streaming=False,
             )



documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://192.168.48.29/query/api-documentation/features"]
)

local_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=local_embeddings)
index = SummaryIndex.from_documents(documents, service_context=service_context)

# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("Make a list of aggregators?")

display(Markdown(f"<b>{response}</b>"))

